## **Data Warehouse Loader Notebook**

This notebook loads data into SQLite schema. This approach is being used to store and access data more efficiently than pure pandas dataframes.

#### ***Environment Configuration***

Helper python file to proper handle db integration

In [1]:
# Setup project environment
import sys
from pathlib import Path

# Import setup_project
sys.path.append(str(Path.cwd()))
from setup_project import setup_environment

# Run setup
paths = setup_environment()

db_path = paths['DB_PATH']
raw_layer = paths['RAW_DATA_DIR']

✅ Environment configured successfully!


#### ***Data Warehouse Connection***
Function to check if data warehouse is available, recreate if not.

In [2]:
from sqlalchemy import create_engine, inspect, MetaData

# Create database file if missing
if not db_path.exists():
    print(f"Database not found at {db_path}, creating a new one...")
    db_path.parent.mkdir(parents=True, exist_ok=True)
    db_path.touch()

engine = create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = inspect(engine)
metadata = MetaData()

print(f"✅ Connected to database at {db_path}")

✅ Connected to database at /home/falatfernando/Desktop/bdq_resistance_study/mtb_resistance_db/mtb_warehouse.db


#### ***Data Warehouse Quick Overview***

Just a little debug step so I can know the db content and if things are correctly connected.

In [3]:
# Get all table names
table_names = inspector.get_table_names()
print("Tables in database:", table_names)

# Get schema information for a specific table
for table_name in table_names:
    columns = inspector.get_columns(table_name)
    print(f"\nColumns in {table_name}:")
    for column in columns:
        print(f"  {column['name']}: {column['type']}")

Tables in database: ['ref_genome_annotation', 'reference_genome', 'sample_genome']

Columns in ref_genome_annotation:
  id: INTEGER
  genome_type: VARCHAR
  gene_name: VARCHAR
  start_pos: INTEGER
  end_pos: INTEGER
  strand: VARCHAR
  product: VARCHAR

Columns in reference_genome:
  id: INTEGER
  genome_type: VARCHAR
  chromosome: VARCHAR
  position: INTEGER
  base: VARCHAR

Columns in sample_genome:
  id: INTEGER
  genome_type: VARCHAR
  sample_name: VARCHAR
  chromosome: VARCHAR
  position: INTEGER
  base: VARCHAR
  reference_id: INTEGER


####  ***Actual Schema Table Materialization*** ⚠️

Those cells actually materializes tables into the connected SQLite Data Warehouse. Be careful when running!

In [4]:
# Handling paths for SQLAlchemy modules imports
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

##### **Populating ReferenceGenome table with H37Rv reference genome from raw layer**

In [5]:
# Reference Sequence
import gzip
from mtb_resistance_db.database import session
from mtb_resistance_db.models import ReferenceGenome

def load_fasta_gz_bulk(filepath, batch_size=10000):
    batch = []
    chrom = None
    pos = 0

    with gzip.open(filepath, 'rt') as fh:
        for line in fh:
            line = line.strip()
            if line.startswith('>'):
                chrom = line[1:]
                pos = 1
            else:
                for base in line:
                    batch.append({
                        'genome_type': 'H37Rv reference',
                        'chromosome': chrom,
                        'position': pos,
                        'base': base
                    })
                    pos += 1

                    # Once batch_full, flush to DB
                    if len(batch) >= batch_size:
                        session.bulk_insert_mappings(ReferenceGenome, batch)
                        session.commit()
                        batch.clear()

    # Insert any remainder
    if batch:
        session.bulk_insert_mappings(ReferenceGenome, batch)
        session.commit()

In [6]:
raw_reference_fasta_path = os.path.join(raw_layer, 'GCF_000195955.2_ASM19595v2_genomic.fna.gz')
load_fasta_gz_bulk(raw_reference_fasta_path)

##### **Populating GeneAnnotation table with H37Rv reference annotations from raw layer**

In [7]:
import gzip
from mtb_resistance_db.database import session
from mtb_resistance_db.models import GeneAnnotation

def load_gtf_gz(filepath):
    batch = []  # Prepare a batch list to accumulate data
    with gzip.open(filepath, 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue  # Skip comments
            parts = line.strip().split('\t')
            if len(parts) >= 9:
                genome_type = parts[0]
                feature = parts[2]
                start = int(parts[3])
                end = int(parts[4])
                strand = parts[6]
                attributes = parts[8]
                
                # Extract gene_name from attributes
                gene_name = None
                for attribute in attributes.split(';'):
                    if 'gene_name' in attribute:
                        gene_name = attribute.split('"')[1]
                        break
                
                if gene_name and feature == 'gene':
                    batch.append({
                        'genome_type': genome_type,
                        'gene_name': gene_name,
                        'start_pos': start,
                        'end_pos': end,
                        'strand': strand,
                        'product': None  # You can parse 'product' if available
                    })

                # Commit in batches of 1000 to prevent memory issues and improve speed
                if len(batch) >= 1000:
                    session.bulk_insert_mappings(GeneAnnotation, batch)
                    session.commit()
                    batch.clear()  # Clear the batch after commit

    # Final commit for any remaining data in the batch
    if batch:
        session.bulk_insert_mappings(GeneAnnotation, batch)
        session.commit()

In [8]:
raw_reference_gtf_path = os.path.join(raw_layer, 'GCF_000195955.2_ASM19595v2_genomic.gtf.gz')
load_gtf_gz(raw_reference_gtf_path)